In [1]:
!git clone https://github.com/ChaitaliV/Location-Extractor

Cloning into 'Location-Extractor'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 269 (delta 74), reused 108 (delta 59), pack-reused 129
Receiving objects: 100% (269/269), 503.32 KiB | 8.99 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install -q transformers
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
from IPython.display import Image
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import AdamW, BertForTokenClassification, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import re

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
df = pd.read_csv('Location-Extractor/Data/places_data.csv')
df.head()

,Unnamed: 0.4,text,name,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index
0,0,"Panchvati Fabric Store,\nShivam Garments,\nShr...","['Panchvati Fabric Store', 'Shivam Garments', ...",NaN,NaN,NaN,NaN,NaN
1,1,Vesu is a trendy and upscale area in Surat tha...,"['Aura Designer Boutique', 'En Vogue']",NaN,NaN,NaN,NaN,NaN
2,2,"For those looking for Western-style clothing, ...","['Blush', 'Zohra Fashion', 'Aanchal Boutique',...",NaN,NaN,NaN,NaN,NaN
3,3,Ghod Dod Road is a bustling shopping destinati...,"['Ghod Dod Road', 'Kalki Fashion', 'Saree Pala...",NaN,NaN,NaN,NaN,NaN
4,4,For those looking for contemporary Western wea...,"['The Dressing Room', 'Seasons', 'Jinal Fashio...",NaN,NaN,NaN,NaN,NaN


In [7]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1','index','Unnamed: 0.2','Unnamed: 0.3','Unnamed: 0.4'], axis=1)

In [8]:
df['name'][0]

"['Panchvati Fabric Store', 'Shivam Garments', 'Shree Ganesham Textiles', 'Raghukul Textiles', 'Geetanjali Fashion Studio', 'New Fashion Center', 'Poonam Sarees', 'Kajal Sarees', 'Tulsi Fashion', 'Ankit Garments']"

In [9]:
new_list = []
all_places = df['name'][:]
for data in all_places:
  l = data[1:-1].split(',')
  final_list = []
  for place in l:
    if place[0] == ' ':
      final_list.append(place[2:-1])
    else :
      final_list.append(place[1:-1])
  new_list.append(final_list)

In [10]:
df['name'] = new_list

In [11]:
df

,text,name
0,"Panchvati Fabric Store,\nShivam Garments,\nShr...","[Panchvati Fabric Store, Shivam Garments, Shre..."
1,Vesu is a trendy and upscale area in Surat tha...,"[Aura Designer Boutique, En Vogue]"
2,"For those looking for Western-style clothing, ...","[Blush, Zohra Fashion, Aanchal Boutique, Fabin..."
3,Ghod Dod Road is a bustling shopping destinati...,"[Ghod Dod Road, Kalki Fashion, Saree Palace]"
4,For those looking for contemporary Western wea...,"[The Dressing Room, Seasons, Jinal Fashion, Di..."
...,...,...
1877,I recently took a trip to Bordeaux and was blo...,"[Le Chapon Fin, Garopapilles, La Tupina, Maiso..."
1878,I recently took a trip to Paris and was amazed...,"[L'Ambroisie, Septime, Chez Paul, Ladurée]"
1879,"On my last visit to Lyon, I was blown away by ...","[Chez Hugon, Paul Bocuse, Fromagerie Mons]"
1880,Paris is known for its romantic atmosphere and...,"[Le Jules Verne, Chez L'Ami Jean, Guy Savoy]"


In [12]:
# Creating sentence, label lists and adding Bert tokens
sentences = df.text.values

# Adding CLS and SEP tokens at the beginning and end of each sentence for BERT
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

In [13]:
#@title Activating the BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
MAX_LEN = 512

In [14]:
input_ids = [tokenizer.encode(x,add_special_tokens=True) for x in sentences]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [15]:
def tokenize_label(text, labels):
  # Tokenize the input text
  tokenized_text = tokenizer.encode(text, add_special_tokens=True)
  # Initialize the label list with "O" (outside) label for each token
  token_labels = ["O"] * len(tokenized_text)

  # Find the indices of the restaurant name tokens in the input text
  for label in labels:
    # Tokenize the restaurant name and get its token IDs
    tokenized_restaurant_name = tokenizer.encode(label, add_special_tokens=True)
    restaurant_name_ids = tokenized_restaurant_name[1:-1]  # exclude the [CLS] and [SEP] tokens

    for i in range(len(tokenized_text) - len(restaurant_name_ids) + 1):
        if tokenized_text[i:i+len(restaurant_name_ids)] == restaurant_name_ids:
            # Mark the restaurant name tokens with the "B-RESTAURANT" (beginning) label
            token_labels[i] = "B-RESTAURANT"
            for j in range(i+1, i+len(restaurant_name_ids)):
                # Mark the remaining tokens of the restaurant name with the "I-RESTAURANT" (inside) label
                token_labels[j] = "I-RESTAURANT"

  # Print the tokenized input text and corresponding labels
  return token_labels

In [16]:
input_labels = []
for p in range (0,len(df)):
  text = df['text'][p]
  label = df['name'][p]
  label = tokenize_label(text, label)
  input_labels.append(label)

In [17]:
# Convert the labels to label IDs
label_ids = []
for label_list in input_labels:
    label_ids.append([2 if label == "B-RESTAURANT" else 3 if label == "I-RESTAURANT" else 1 for label in label_list])
label_ids = pad_sequences(label_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [18]:
#Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
train_ids = input_ids
train_label = label_ids
train_attention_masks = attention_masks

In [19]:
len(train_attention_masks[0])

512

In [20]:
# Splitting data into train and validation sets
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_ids, train_label,random_state = 2018, test_size = 0.1 )
train_masks, validation_masks, _, _ = train_test_split(train_attention_masks, train_ids,
                                             random_state=2018, test_size=0.1)
     

In [21]:
train_labels

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 2, 3, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [22]:
#Converting all the data into torch tensors
# Torch tensors are the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
print(type(train_inputs))

<class 'torch.Tensor'>


In [23]:
# Selecting a Batch Size and Creating and Iterator
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 4

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
print(len(train_inputs))
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
     

1693


In [24]:
# Bert Model Configuration
# Initializing a BERT bert-base-uncased style configuration
# Transformer Installation
try:
  import transformers
except:
  print("Installing transformers")
  !pip -qq install transformers
  
from transformers import BertModel, BertConfig
configuration = BertConfig()

# Initializing a model from the bert-base-uncased style configuration
# This process will load only configuration and not the weights associated with the model
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config
print(configuration)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [25]:
# Loading the Hugging Face Bert Uncased Base Model 
model = BertForTokenClassification.from_pretrained("bert-base-cased",num_labels=len(df),
    output_attentions = False,
    output_hidden_states = False)
model = nn.DataParallel(model)
model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

DataParallel(
  (module): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bia

In [26]:
# Optimizer Grouped Parameters
# reference:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L102


# Don't apply weight decay to any parameters whose names include these tokens.
# (Here, the BERT doesn't have `gamma` or `beta` parameters, only `bias` terms)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']
# Separate the `weight` parameters from the `bias` parameters. 
# - For the `weight` parameters, this specifies a 'weight_decay_rate' of 0.01. 
# - For the `bias` parameters, the 'weight_decay_rate' is 0.0. 
optimizer_grouped_parameters = [
    # Filter for all parameters which *don't* include 'bias', 'gamma', 'beta'.
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.1},
    
    # Filter for parameters which *do* include those.
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
# Note - `optimizer_grouped_parameters` only includes the parameter values, not 
# the names.

In [27]:
# The Hyperparameters for the Training Loop 

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

optimizer = AdamW(optimizer_grouped_parameters,
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )
# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0.1, # Default value in run_glue.py
                                            num_training_steps = total_steps)
     

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
#Creating the Accuracy Measurement Function
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [29]:
# The Training Loop
t = [] 

# Store our loss and accuracy for plotting
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs['loss']
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()

    # Update the learning rate.
    scheduler.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = np.argmax(logits[0].to('cpu').numpy(), axis=2)
    label_ids = b_labels.to('cpu').numpy()
    #print(logits == label_ids)

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Train loss: 0.3043407210948403


Epoch:  33%|███▎      | 1/3 [03:02<06:05, 182.77s/it]

Validation Accuracy: 0.9966227213541666
Train loss: 0.019365046253506938


Epoch:  67%|██████▋   | 2/3 [06:08<03:04, 184.68s/it]

Validation Accuracy: 0.9982096354166666
Train loss: 0.011994587697356692


Epoch: 100%|██████████| 3/3 [09:14<00:00, 184.73s/it]

Validation Accuracy: 0.9985453287760416


In [30]:
torch.save(model,r'/content/drive/MyDrive/BERT_text.pt')

In [31]:
model = torch.load(r'/content/drive/MyDrive/BERT_text.pt')

In [ ]:
def clean_string(list):
  l = []
  for ele in list:
    if ele == ' ' or ele == '':
      pass
    else:
      l.append(ele)
  return l

def add_suffix(p):
  match = re.search(p,sent)
  # Define a list of categories to check against
  categories = ['Restaurant', 'Cafe', 'House', 'Bar', 'Pub', 'Kitchen','Club','Bakery','Shop','Room','Shack', 'Palace','Hill','Garden', 'Street']
  if match:
    for category in categories:
        if match.end() < len(sent) and sent[match.end():].lower().startswith(category.lower()):
            l = match.group() + '' + category
            return l
            break
    return match.group()
      
  else:
    return p

In [ ]:
def extract_location(sent):
  """ extrecting location """
  true_label = []
  tokenized_sentence = tokenizer.encode(sent)
  input_ids = torch.tensor([tokenized_sentence])
  with torch.no_grad():
    output = model(input_ids)
  label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
  tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, label_indices[0]):
      if token.startswith("##"):
          new_tokens[-1] = new_tokens[-1] + token[2:]
      else:
          new_labels.append(label_idx)
          new_tokens.append(token)
  for token, label in zip(new_tokens, new_labels):
    if (label==3):
      true_label.append(token)
    else:
      true_label.append('#')
  
  # Join the words into a single string
  label = " ".join(true_label)
  label = label.replace(" ' s","'s")
  l = label.split('#')
  p = clean_string(l)

  """  Adding suffixes """
  final_list = []
  for ele in p:
    final_list.append(add_suffix(ele))
  
  return final_list


In [ ]:
sent = "which houses a variety of rare plants and trees,the historic Bangalore Palace"

In [ ]:
extract_location(sent)

[' Bangalore Palace']

In [ ]:
def multiline_data(text):
  names = []
  l = text.split(".")
  for sent in l:
    names.append(extract_location(sent))
  return list(filter(None,names))


In [ ]:
sent = """Bangalore, also known as Bengaluru, is the capital city of the Indian state of Karnataka. 
It is a hub for technology and start-up companies, 
and also has a rich history and culture. Some of the popular places to visit in Bangalore
 include the iconic Lalbagh Botanical Garden, which houses a variety of rare plants and trees,
  the historic Bangalore Palace, and the Tipu Sultan's Summer Palace, a stunning example of Indo-Islamic architecture.
   The city is also home to several museums, including the Government Museum and the National Gallery of Modern Art.
    For a taste of local life, visitors can explore the bustling markets of Commercial Street or sample the delicious street 
    food at the lively VV Puram Food Street. Other notable attractions include the serene Ulsoor Lake and the picturesque Nandi Hills."""
city = 'Banglore'

In [ ]:
multiline_data(sent)

[[' Lalbagh Botanical Garden',
  ' Bangalore Palace',
  " Tipu Sultan's Summer Palace"],
 [' Government '],
 [' Commercial Street', ' VV Puram Food Street'],
 [' Ulsoor ', ' Nandi Hill']]